In [ ]:
# !git clone https://github.com/marcusklasson/GroceryStoreDataset.git

### Libraries

In [1]:
import pandas as pd
import numpy as np
import torch
from torch import nn, optim
from torch.autograd import Variable
import torch.functional as F
from torch.optim import lr_scheduler
from torch.utils.data import Dataset
from torchvision.io import read_image
from torch.utils.data import Sampler
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.models import vgg19, resnet18
from torchvision import models, transforms
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder

import os
import copy
import tqdm
import random
import time

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Utils

In [ ]:
def seed_everything(seed: int):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [ ]:
use_gpu = torch.cuda.is_available()
if use_gpu:
    print("Using CUDA")

Using CUDA


### EDA

In [ ]:
base_dir = r'/content/drive/MyDrive/Colab Notebooks/few shot learning/GroceryStoreDataset/dataset'

classes = pd.read_csv(f'{base_dir}/classes.csv') # 43 classes
train = pd.read_csv(f'{base_dir}/train.txt', header=None, names=['location', 'class_id', 'coarse_class_id'])
val = pd.read_csv(f'{base_dir}/val.txt', header=None, names=['location', 'class_id', 'coarse_class_id'])
test = pd.read_csv(f'{base_dir}/test.txt', header=None, names=['location', 'class_id', 'coarse_class_id'])

In [ ]:
print(train.groupby('coarse_class_id').location.count().min(), val.groupby('coarse_class_id').location.count().min(), test.groupby('coarse_class_id').location.count().min())
print(train.shape, val.shape, test.shape)
# train

16 3 14
(2640, 3) (296, 3) (2485, 3)


In [ ]:
train['type'] = train.location.apply(lambda x: (x.split('/')[1]))
train.type.value_counts()
for t in train.type.unique():
  current_classes = train[train.type==t].coarse_class_id.unique()
  selected_ids = np.random.choice(current_classes, 5, replace=False)
  print(t)
  print(selected_ids)
  print(classes[classes[classes.columns[3]].isin(selected_ids)][classes.columns[2]].unique())
# classes

Fruit
[ 1  4  0 10  6]
['Apple' 'Avocado' 'Lemon' 'Mango' 'Papaya']
Packages
[26 24 27 23 22]
['Oat-Milk' 'Sour-Cream' 'Sour-Milk' 'Soy-Milk' 'Yoghurt']
Vegetables
[42 41 33 34 30]
['Cabbage' 'Garlic' 'Ginger' 'Tomato' 'Zucchini']


### Data classes

In [ ]:
# Dataset class
class few_shot_dataset(Dataset):
  def __init__(self, directory, skip_classes=None, transform=None, target_transform=None, general=True, specific_subclass=None, max_classes=10 , datatype='train', le=None) -> None:# , train=True, val=False, test=False) -> None:
    super().__init__()
    assert (general and not specific_subclass) or ((specific_subclass and not general )), 'one of general or specific_subclass must be false'
    self.base_directory = directory
    self.skip_classes = skip_classes
    self.specific_subclass = specific_subclass
    self.general = general
    self.transform = transform
    self.target_transform = target_transform
    self.max_classes = max_classes
    self.extra_skip_classes = None
    self.datatype = datatype

    self.classes = pd.read_csv(f'{directory}/classes.csv')
    self.data = pd.read_csv(f'{directory}/{datatype}.txt', header=None, names=['location', 'class_id', 'coarse_class_id'])
    # self.val_df = pd.read_csv(f'{directory}/val.txt', header=None, names=['location', 'class_id', 'coarse_class_id'])
    # self.test_df = pd.read_csv(f'{directory}/test.txt', header=None, names=['location', 'class_id', 'coarse_class_id'])
    
    # self.train = pd.concat([self.train_df, self.val_df, self.test_df]).copy()

    self.data['type'] = self.data.location.apply(lambda x: (x.split('/')[1]))
    if specific_subclass:
      self.data = self.data[self.data.type==specific_subclass]
    if not self.skip_classes :
      # filter = self.data.coarse_class_id.isin(skip_classes)
      # filter = (~filter)
      # self.data = self.data[filter]
    # else:
      all_classes = self.data.coarse_class_id.unique()
      self.skip_classes = np.random.choice(all_classes, 5, replace=False)
    
    filter = self.data.coarse_class_id.isin(self.skip_classes)
    filter = (~filter) # &(self.data[self.data.type==specific_subclass]) if ((not general) and specific_subclass) else (~filter)
    self.data = self.data[filter].reset_index(drop=True)

    if self.max_classes:
      if self.max_classes< self.data.coarse_class_id.nunique():
        all_classes = self.data.coarse_class_id.unique()
        self.extra_skip_classes = np.random.choice(all_classes, self.data.coarse_class_id.nunique()-self.max_classes, replace=False)
        self.skip_classes = np.concatenate([self.skip_classes, self.extra_skip_classes])

        filter = self.data.coarse_class_id.isin(self.skip_classes)
        filter = (~filter) # &(self.data[self.data.type==specific_subclass]) if ((not general) and specific_subclass) else (~filter)
        self.data = self.data[filter].reset_index(drop=True)

    if self.datatype=='train':
      self.le = LabelEncoder()
      self.data['label'] = self.le.fit_transform(self.data.coarse_class_id)
    else:
      assert le is not None
      self.le = le
      self.data['label'] = self.le.transform(self.data.coarse_class_id)

    self.data = self.data.copy()
  
  def __len__(self):
    return self.data.shape[0]

  def __getitem__(self, idx):
    img_path = f"{self.base_directory}/{self.data.loc[idx, 'location']}"
    image = read_image(img_path)
    label = self.data.loc[idx, 'label']
    if self.transform:
        image = self.transform(image)
    if self.target_transform:
        label = self.target_transform(label)
    return image, label

In [ ]:
# len(dataset)==train[(train.type=='Fruit')&(~train.coarse_class_id.isin([1, 3, 4, 9, 5]))].shape[0]
# dataset.train.equals(train[(train.type=='Fruit')&(~train.coarse_class_id.isin([1, 3, 4, 9, 5]))].reset_index(drop=True))
# dataset.train.iloc[278]
# dataset.train.coarse_class_id.nunique(), dataset.train.coarse_class_id.unique()
# dataset.data.coarse_class_id.value_counts()


### Helper functions

In [ ]:
def eval_model(vgg, criterion):
    since = time.time()
    avg_loss = 0
    avg_acc = 0
    loss_test = 0
    acc_test = 0
    test_batches = len(dataloaders[TEST])
    print("Evaluating model")
    print('-' * 10)
    for i, data in enumerate(dataloaders[TEST]):
        if i % 100 == 0:
            print("\rTest batch {}/{}".format(i, test_batches), end='', flush=True)
        vgg.train(False)
        vgg.eval()
        inputs, labels = data
        if use_gpu:
            inputs, labels = Variable(inputs.cuda(), volatile=True), Variable(labels.cuda(), volatile=True)
        else:
            inputs, labels = Variable(inputs, volatile=True), Variable(labels, volatile=True)
        outputs = vgg(inputs)
        _, preds = torch.max(outputs.data, 1)
        loss = criterion(outputs, labels)
        loss_test += loss.data # loss.data[0]
        acc_test += torch.sum(preds == labels.data)
        del inputs, labels, outputs, preds
        torch.cuda.empty_cache()
    avg_loss = loss_test / dataset_sizes[TEST]
    avg_acc = acc_test / dataset_sizes[TEST]
    elapsed_time = time.time() - since
    print()
    print("Evaluation completed in {:.0f}m {:.0f}s".format(elapsed_time // 60, elapsed_time % 60))
    print("Avg loss (test): {:.4f}".format(avg_loss))
    print("Avg acc (test): {:.4f}".format(avg_acc))
    print('-' * 10)

### Loading data

In [ ]:
seed = 42
seed_everything(seed)

In [ ]:
TRAIN = 'train'
VAL = 'val'
TEST = 'test'

image_size = 224 # 512 if torch.cuda.is_available() else 128
skip_classes=[1, 3, 4, 9, 5]

In [ ]:
# Transforms
data_transforms = {
    TRAIN: transforms.Compose([
        transforms.ToPILImage(),
        transforms.RandomResizedCrop(image_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ]),
    VAL: transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(256),
        transforms.CenterCrop(image_size),
        transforms.ToTensor(),
    ]),
    TEST: transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(256),
        transforms.CenterCrop(image_size),
        transforms.ToTensor(),
    ])
}

In [ ]:
# datasets
image_datasets  = {}
image_datasets[TRAIN] = few_shot_dataset(directory=base_dir, skip_classes=skip_classes, specific_subclass='Fruit', general=False, datatype=TRAIN, transform=data_transforms[TRAIN])
image_datasets[VAL] = few_shot_dataset(directory=base_dir, skip_classes=list(image_datasets[TRAIN].skip_classes), specific_subclass='Fruit', general=False, datatype=VAL, transform=data_transforms[VAL], le=image_datasets[TRAIN].le)
image_datasets[TEST] = few_shot_dataset(directory=base_dir, skip_classes=list(image_datasets[TRAIN].skip_classes), specific_subclass='Fruit', general=False, datatype=TEST, transform=data_transforms[TEST], le=image_datasets[TRAIN].le)

In [ ]:
# dataloaders
dataloaders = {
    x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=0) for x in [TRAIN, VAL, TEST]
    }

In [ ]:
dataset_sizes = {x: len(image_datasets[x]) for x in [TRAIN, VAL, TEST]}
dataset_sizes

{'train': 561, 'val': 49, 'test': 545}

In [ ]:
total_classes = image_datasets[TRAIN].data.coarse_class_id.nunique()

### Model

First step (optional) is to take a network (pretrained if possible), and train it on a dataset with similar classes. In this case, the main dataset had 3 distinct classes - fruits, packages and vegetables. 
Fruit class has been selected. From the training data, all fruit images were selected, and the vgg19 model, with imagenet weights, was further trained on this dataset. The test cases, or the images which we need to recognize, will not be present in this dataset.
Depending on the use case, this finetuning of the model can be done on more relevant datasets.


In [ ]:
def load_vgg19_model(pretrained):
  vgg19 = models.vgg19(pretrained=pretrained)
  for param in vgg19.features.parameters():
    param.require_grad = False
  num_features = vgg19.classifier[6].out_features
  features = list(vgg19.classifier.children()) # Remove last layer
  features.extend([nn.Dropout(0.5), nn.Linear(num_features, total_classes)]) # Add our layer with 4 outputs
  vgg19.classifier = nn.Sequential(*features)
  if use_gpu:
    vgg19.cuda()
  return vgg19


In [ ]:
vgg19 = load_vgg19_model(pretrained=True)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
print(vgg19)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [ ]:
 #.cuda() will move everything to the GPU side
criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.SGD(vgg19.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

### Evaluating model accuracy before training

In [ ]:
print("Test before training")
eval_model(vgg19, criterion)

Test before training
Evaluating model
----------
Test batch 0/18

<ipython-input-13-d66d1c47deeb>:17: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  inputs, labels = Variable(inputs.cuda(), volatile=True), Variable(labels.cuda(), volatile=True)



Evaluation completed in 0m 10s
Avg loss (test): 0.1023
Avg acc (test): 0.0954
----------


### Training the model using the 10 selected image classes

In [ ]:
def train_model(vgg, criterion, optimizer, scheduler, num_epochs=10):
    since = time.time()
    best_model_wts = copy.deepcopy(vgg.state_dict())
    best_acc = 0.0
    avg_loss = 0
    avg_acc = 0
    avg_loss_val = 0
    avg_acc_val = 0
    train_batches = len(dataloaders[TRAIN])
    val_batches = len(dataloaders[VAL])
    
    for epoch in range(num_epochs):
        print("Epoch {}/{}".format(epoch, num_epochs))
        print('-' * 10)
        loss_train = 0
        loss_val = 0
        acc_train = 0
        acc_val = 0
        vgg.train(True)        
        for i, data in enumerate(dataloaders[TRAIN]):
            if i % 100 == 0:
                print("\rTraining batch {}/{}".format(i, train_batches), end='', flush=True)
            # Use half training dataset
            # if i >= train_batches / 2:
            #     break
            inputs, labels = data
            if use_gpu:
                inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
            else:
                inputs, labels = Variable(inputs), Variable(labels)
            optimizer.zero_grad()
            outputs = vgg(inputs)
            _, preds = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            loss_train += loss.data # loss.data[0]
            acc_train += torch.sum(preds == labels.data)
            del inputs, labels, outputs, preds
            torch.cuda.empty_cache()
        
        print()
        # * 2 as we only used half of the dataset
        avg_loss = loss_train/ dataset_sizes[TRAIN]
        avg_acc = acc_train/ dataset_sizes[TRAIN]
        
        vgg.train(False)
        vgg.eval()
            
        for i, data in enumerate(dataloaders[VAL]):
            if i % 100 == 0:
                print("\rValidation batch {}/{}".format(i, val_batches), end='', flush=True)
            inputs, labels = data
            if use_gpu:
                inputs, labels = Variable(inputs.cuda(), volatile=True), Variable(labels.cuda(), volatile=True)
            else:
                inputs, labels = Variable(inputs, volatile=True), Variable(labels, volatile=True)
            optimizer.zero_grad()
            outputs = vgg(inputs)
            _, preds = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)
            loss_val += loss.data # loss.data[0]
            acc_val += torch.sum(preds == labels.data)
            del inputs, labels, outputs, preds
            torch.cuda.empty_cache()
        avg_loss_val = loss_val / dataset_sizes[VAL]
        avg_acc_val = acc_val / dataset_sizes[VAL]
        print()
        print("Epoch {} result: ".format(epoch))
        print("Avg loss (train): {:.4f}".format(avg_loss))
        print("Avg acc (train): {:.4f}".format(avg_acc))
        print("Avg loss (val): {:.4f}".format(avg_loss_val))
        print("Avg acc (val): {:.4f}".format(avg_acc_val))
        print('-' * 10)
        print()
        if avg_acc_val > best_acc:
            best_acc = avg_acc_val
            best_model_wts = copy.deepcopy(vgg.state_dict())
    elapsed_time = time.time() - since
    print()
    print("Training completed in {:.0f}m {:.0f}s".format(elapsed_time // 60, elapsed_time % 60))
    print("Best acc: {:.4f}".format(best_acc))
    
    vgg.load_state_dict(best_model_wts)
    return vgg

In [ ]:
vgg19 = train_model(vgg19, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=50)
torch.save(vgg19.state_dict(), '/content/drive/MyDrive/Colab Notebooks/few shot learning/VGG19_few_shot.pt')

Epoch 0/50
----------
Training batch 0/18
Validation batch 0/2

<ipython-input-24-d97f060ce73c>:55: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  inputs, labels = Variable(inputs.cuda(), volatile=True), Variable(labels.cuda(), volatile=True)



Epoch 0 result: 
Avg loss (train): 0.0613
Avg acc (train): 0.4421
Avg loss (val): 0.0419
Avg acc (val): 0.6327
----------

Epoch 1/50
----------
Training batch 0/18
Validation batch 0/2
Epoch 1 result: 
Avg loss (train): 0.0288
Avg acc (train): 0.6916
Avg loss (val): 0.0348
Avg acc (val): 0.7143
----------

Epoch 2/50
----------
Training batch 0/18
Validation batch 0/2
Epoch 2 result: 
Avg loss (train): 0.0168
Avg acc (train): 0.8146
Avg loss (val): 0.0406
Avg acc (val): 0.7143
----------

Epoch 3/50
----------
Training batch 0/18
Validation batch 0/2
Epoch 3 result: 
Avg loss (train): 0.0093
Avg acc (train): 0.9127
Avg loss (val): 0.0409
Avg acc (val): 0.7347
----------

Epoch 4/50
----------
Training batch 0/18
Validation batch 0/2
Epoch 4 result: 
Avg loss (train): 0.0091
Avg acc (train): 0.9002
Avg loss (val): 0.0244
Avg acc (val): 0.7551
----------

Epoch 5/50
----------
Training batch 0/18
Validation batch 0/2
Epoch 5 result: 
Avg loss (train): 0.0090
Avg acc (train): 0.9127
Avg

In [ ]:
eval_model(vgg19, criterion)

Evaluating model
----------
Test batch 0/18

<ipython-input-13-d66d1c47deeb>:17: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  inputs, labels = Variable(inputs.cuda(), volatile=True), Variable(labels.cuda(), volatile=True)



Evaluation completed in 0m 6s
Avg loss (test): 0.0076
Avg acc (test): 0.9138
----------


In [ ]:
eval_model(vgg19, criterion)

Evaluating model
----------
Test batch 0/18

<ipython-input-13-d66d1c47deeb>:17: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  inputs, labels = Variable(inputs.cuda(), volatile=True), Variable(labels.cuda(), volatile=True)



Evaluation completed in 0m 6s
Avg loss (test): 0.0076
Avg acc (test): 0.9138
----------


### Loading back the model

In [ ]:
vgg19 = load_vgg19_model(pretrained=False)
vgg19.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/few shot learning/VGG19_few_shot.pt'))
vgg19.eval()

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [ ]:
vgg19_dup = copy.deepcopy(vgg19)

In [ ]:
features = list(vgg19.classifier.children())[:-2] # Remove last layer
# features.extend([nn.Dropout(0.5), nn.Linear(num_features, total_classes)]) # Add our layer with 4 outputs
vgg19.classifier = nn.Sequential(*features)
if use_gpu:
  vgg19.cuda()

In [ ]:
vgg19.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)

### few shot learning results

In [ ]:
image_datasets[TRAIN].skip_classes

array([ 1,  3,  4,  9,  5, 14, 16,  0, 17])

In [ ]:
# Creating a dataset class to load images
class nWayKShotData(Dataset):
  def __init__(self, n, k, q, directory, transform=None, target_transform=None, datatype='train', classes_to_sample_from=None) -> None:# , train=True, val=False, test=False) -> None:
     super().__init__()
     self.n_way = n
     self.k_shot = k
     self.n_query = q
     self.base_directory = directory
     self.transform = transform
     self.target_transform = target_transform
     self.classes_to_sample_from = classes_to_sample_from
     assert self.n_way == len(self.classes_to_sample_from)

     self.classes = pd.read_csv(f'{directory}/classes.csv')
     self.data = pd.read_csv(f'{directory}/{datatype}.txt', header=None, names=['location', 'class_id', 'coarse_class_id']) 
     self.data = self.data[self.data.coarse_class_id.isin(self.classes_to_sample_from)].reset_index().copy()
  
  def __len__(self):
    return self.data.shape[0]

  def __getitem__(self, idx):
    img_path = f"{self.base_directory}/{self.data.loc[idx, 'location']}"
    image = read_image(img_path)
    label = self.data.loc[idx, 'coarse_class_id']
    if self.transform:
        image = self.transform(image)
    if self.target_transform:
        label = self.target_transform(label)
      
    # if use_gpu:
    #   image, label = Variable(image.cuda()), Variable(label.cuda())
    # else:
    #   image, label = Variable(image), Variable(label)
    return image, label

In [ ]:
# Creating a data loader class to load datra in correct format
# This will select pre-specified or random classes, and return 
# k support images of
# n classes, and 
# q query images
# n times

class nWayKShotSampler(Sampler):
  def __init__(self, n, k, q, dataset, n_times, classes_to_sample_from=None, ) -> None:
     self.n_way = n
     self.k_shot = k
     self.n_query = q
     self.dataset = dataset
     self.n_times = n_times
     self.classes_to_sample_from = classes_to_sample_from
     
     print(f'classes selected - {self.classes_to_sample_from}')
     self.relevant_images = dict.fromkeys(self.classes_to_sample_from)
     for image_class in self.relevant_images.keys():
       self.relevant_images[image_class] = self.dataset.data[self.dataset.data.coarse_class_id==image_class].index.to_list()

  def __iter__(self):
    for _ in range(self.n_times):
      ids = np.concatenate([
          np.random.choice(
              self.relevant_images[z], 
              self.k_shot + self.n_query,
              replace=False if len(self.relevant_images[z]) > self.k_shot + self.n_query else True)
           for z in self.relevant_images.keys()]).tolist()
      yield ids

  def __len__(self):
    return self.n_times

  # Function defining how to collate the images generated while loading data
  def collate(self, input_data):
    all_images = torch.cat([subset[0].unsqueeze(0) for subset in input_data])
    all_images = all_images.reshape(self.n_way, self.k_shot + self.n_query, *all_images.shape[1:])
    support_images = all_images[:, :self.k_shot].reshape(-1, *all_images.shape[2:]).squeeze()
    query_images = all_images[:, self.k_shot:].reshape(-1, *all_images.shape[2:]).squeeze()
    all_labels = torch.tensor([subset[1] for subset in input_data]).reshape(self.n_way, self.k_shot + self.n_query)

    diction = dict.fromkeys(all_labels.unique())
    for i, val in enumerate(diction.keys()):
      diction[val]=i
      all_labels[all_labels==val] = diction[val]
    support_labels = all_labels[:, :self.k_shot].flatten()
    query_labels = all_labels[:, self.k_shot:].flatten()
    
    if use_gpu:
      support_images, support_labels, query_images, query_labels = Variable(support_images.cuda()), Variable(support_labels.cuda()), Variable(query_images.cuda()), Variable(query_labels.cuda())
    else:
      support_images, support_labels, query_images, query_labels = Variable(support_images), Variable(support_labels), Variable(query_images), Variable(query_labels)
    
    return (support_images,
            support_labels,
            query_images,
            query_labels
            )

In [ ]:
def get_loaders(n_way, k_shot, n_query, base_dir, transforms, n_iter, classes_to_sample_from=None, datatype='train'):
  data = nWayKShotData(n=n_way, k=k_shot, q=n_query, directory=base_dir, transform=transforms, datatype=datatype, classes_to_sample_from=classes_to_sample_from)
  sampler = nWayKShotSampler(data.n_way, data.k_shot, data.n_query, data, n_times = 30, classes_to_sample_from = data.classes_to_sample_from)
  data_loader = DataLoader(dataset=data, batch_sampler=sampler, collate_fn=sampler.collate)
  print(f'{datatype} loaded')
  return data_loader, sampler, data

In [ ]:
class cnn(nn.Module):
  def __init__(self, backbone) -> None:
    super().__init__()
    self.backbone = backbone
  
  def forward(self, support, support_labels, query, query_labels):
    unique_labels = torch.unique(support_labels)
    support_vec = self.backbone(support)
    support_vec = torch.cat(
        [
            support_vec[support_labels==label].mean(0).unsqueeze(0) for label in unique_labels
        ]
        )
    query_vec = self.backbone(query)
    classification = torch.cdist(query_vec, support_vec).min(axis=1)[1]
    classification_labels = torch.tensor([unique_labels[z] for z in classification])

    # return classification_labels
    return Variable(classification_labels.cuda())

In [ ]:
def evaluate(backbone, loader):
  model = cnn(backbone)

  total = 0
  correct = 0

  for data in tqdm.tqdm(loader):
    support, support_labels, query, query_labels = data
    pred = model(*data)
    # print('a')
    # print(query_labels.get_device())
    # print('b')
    # print(pred.get_device())
    correct += (pred==query_labels).sum()
    total += len(query_labels)
    
  # print(f'\ncorrectly predicted {correct} out of {total} samples')
  # print(f'Accuracy of {100*correct/total}%')
  return correct/total

### Using a pre-trained network, without any training

In [ ]:
all_scores = pd.DataFrame(columns=['n_way', 'k_shot', 'n_query', 'n_iter', 'classes_to_sample_from', 'score'])
all_available_classes = image_datasets[TRAIN].skip_classes

In [ ]:
classes[classes[classes.columns[3]].isin(all_available_classes)][[classes.columns[3], classes.columns[2]]].drop_duplicates()

,Coarse Class ID (int),Coarse Class Name (str)
0,0,Apple
5,1,Avocado
7,3,Kiwi
8,4,Lemon
9,5,Lime
16,9,Orange
23,14,Pineapple
25,16,Pomegranate
26,17,Red-Grapefruit


In [ ]:
for i in range(2, len(all_available_classes)):
  for j in range(2, 7):
    n_way = i #no of classes #3
    k_shot = j #no of samples #5
    n_query = 5
    n_iter = 51 # similar to n_batches
    transform = data_transforms[VAL]
    backbone = copy.deepcopy(vgg19)
    classes_to_sample_from = np.random.choice(all_available_classes, n_way, replace=False)
    data_loader, sampler, data = get_loaders(n_way, k_shot, n_query, base_dir, transform, n_iter, classes_to_sample_from=classes_to_sample_from, datatype=TRAIN)
    score = evaluate(backbone, data_loader)
    all_scores.loc[all_scores.shape[0]] = [n_way, k_shot, n_query, n_iter, classes_to_sample_from, score.cpu().numpy()]
    print(f'n-way {n_way}, k-shot {k_shot}, query {n_query}, iter {n_iter}, classes {classes_to_sample_from} score: {100*score}')

classes selected - [ 1 16]
train loaded


100%|██████████| 30/30 [01:12<00:00,  2.40s/it]


n-way 2, k-shot 2, query 5, iter 51, classes [ 1 16] score: 79.66667175292969
classes selected - [ 3 16]
train loaded


100%|██████████| 30/30 [00:35<00:00,  1.19s/it]


n-way 2, k-shot 3, query 5, iter 51, classes [ 3 16] score: 100.0
classes selected - [9 4]
train loaded


...

n-way 8, k-shot 4, query 5, iter 51, classes [14 16  9  0  4 17  5  3] score: 69.66666412353516
classes selected - [16  1  4  5  0  9 17  3]
train loaded


100%|██████████| 30/30 [00:24<00:00,  1.22it/s]


n-way 8, k-shot 5, query 5, iter 51, classes [16  1  4  5  0  9 17  3] score: 67.41667175292969
classes selected - [17  4  9  3  5 16 14  1]
train loaded


100%|██████████| 30/30 [00:28<00:00,  1.04it/s]

n-way 8, k-shot 6, query 5, iter 51, classes [17  4  9  3  5 16 14  1] score: 77.16667175292969


In [ ]:
all_scores.to_csv('/content/drive/MyDrive/Colab Notebooks/few shot learning/few_shot_untrained_scores_with_training2.csv')

In [ ]:
all_scores

,n_way,k_shot,n_query,n_iter,classes_to_sample_from,score
0,2,2,5,51,"[1, 16]",0.7966667
1,2,3,5,51,"[3, 16]",1.0
2,2,4,5,51,"[9, 4]",0.8833333
3,2,5,5,51,"[14, 0]",0.99
4,2,6,5,51,"[9, 4]",0.8833333
5,3,2,5,51,"[16, 1, 17]",0.87111115
6,3,3,5,51,"[9, 17, 4]",0.7022222
7,3,4,5,51,"[5, 3, 16]",0.90000004
8,3,5,5,51,"[14, 9, 5]",0.9266667
9,3,6,5,51,"[17, 14, 5]",0.9111111


### Performance on just a pretrained network

In [ ]:
all_new_scores = pd.DataFrame(columns=['n_way', 'k_shot', 'n_query', 'n_iter', 'classes_to_sample_from', 'score'])
all_available_classes = image_datasets[TRAIN].skip_classes

In [ ]:
pretrained_vgg = models.vgg19(pretrained=True)
if use_gpu:
  pretrained_vgg.cuda()

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth


  0%|          | 0.00/548M [00:00<?, ?B/s]

In [ ]:
for index, row in all_scores.iterrows():
  n_way = row['n_way']
  k_shot = row['k_shot']
  n_query = row['n_query']
  n_iter = row['n_iter']
  transform = data_transforms[VAL]
  backbone = copy.deepcopy(pretrained_vgg)
  classes_to_sample_from = row['classes_to_sample_from']
  data_loader, sampler, data = get_loaders(n_way, k_shot, n_query, base_dir, transform, n_iter, classes_to_sample_from=classes_to_sample_from, datatype=TRAIN)
  score = evaluate(backbone, data_loader)
  all_new_scores.loc[all_new_scores.shape[0]] = [n_way, k_shot, n_query, n_iter, classes_to_sample_from, score.cpu().numpy()]
  print(f'n-way {n_way}, k-shot {k_shot}, query {n_query}, iter {n_iter}, classes {classes_to_sample_from} score: {100*score}')

classes selected - [ 1 16]
train loaded


100%|██████████| 30/30 [00:05<00:00,  5.97it/s]


n-way 2, k-shot 2, query 5, iter 51, classes [ 1 16] score: 62.33333206176758
classes selected - [ 3 16]
train loaded


100%|██████████| 30/30 [00:05<00:00,  5.29it/s]


n-way 2, k-shot 3, query 5, iter 51, classes [ 3 16] score: 71.0
classes selected - [9 4]
train loaded


...

n-way 8, k-shot 4, query 5, iter 51, classes [14 16  9  0  4 17  5  3] score: 40.66666793823242
classes selected - [16  1  4  5  0  9 17  3]
train loaded


100%|██████████| 30/30 [00:24<00:00,  1.22it/s]


n-way 8, k-shot 5, query 5, iter 51, classes [16  1  4  5  0  9 17  3] score: 32.583335876464844
classes selected - [17  4  9  3  5 16 14  1]
train loaded


100%|██████████| 30/30 [00:26<00:00,  1.12it/s]

n-way 8, k-shot 6, query 5, iter 51, classes [17  4  9  3  5 16 14  1] score: 48.333335876464844


In [ ]:
all_new_scores.to_csv('/content/drive/MyDrive/Colab Notebooks/few shot learning/few_shot_untrained_scores_without_training2.csv')

In [ ]:
all_new_scores

,n_way,k_shot,n_query,n_iter,classes_to_sample_from,score
0,2,2,5,51,"[1, 16]",0.62333333
1,2,3,5,51,"[3, 16]",0.71000004
2,2,4,5,51,"[9, 4]",0.5966667
3,2,5,5,51,"[14, 0]",0.99
4,2,6,5,51,"[9, 4]",0.59333336
5,3,2,5,51,"[16, 1, 17]",0.57111114
6,3,3,5,51,"[9, 17, 4]",0.4577778
7,3,4,5,51,"[5, 3, 16]",0.6111111
8,3,5,5,51,"[14, 9, 5]",0.7777778
9,3,6,5,51,"[17, 14, 5]",0.83555555


In [ ]:
all_scores['classes_to_sample_from'] = all_scores.classes_to_sample_from.apply(lambda x:list(x))
all_new_scores['classes_to_sample_from'] = all_new_scores.classes_to_sample_from.apply(lambda x:list(x))
all_scores['classes_to_sample_from'] = all_scores.classes_to_sample_from.apply(lambda x:str(x))
all_new_scores['classes_to_sample_from'] = all_new_scores.classes_to_sample_from.apply(lambda x:str(x))
all_scores.merge(all_new_scores.rename(columns={'score': 'untrained_score'}), on=['n_way', 'k_shot', 'n_query', 'n_iter', 'classes_to_sample_from'])

,n_way,k_shot,n_query,n_iter,classes_to_sample_from,score,untrained_score
0,2,2,5,51,"[1, 16]",0.7966667,0.62333333
1,2,3,5,51,"[3, 16]",1.0,0.71000004
2,2,4,5,51,"[9, 4]",0.8833333,0.5966667
3,2,5,5,51,"[14, 0]",0.99,0.99
4,2,6,5,51,"[9, 4]",0.8833333,0.59333336
5,3,2,5,51,"[16, 1, 17]",0.87111115,0.57111114
6,3,3,5,51,"[9, 17, 4]",0.7022222,0.4577778
7,3,4,5,51,"[5, 3, 16]",0.90000004,0.6111111
8,3,5,5,51,"[14, 9, 5]",0.9266667,0.7777778
9,3,6,5,51,"[17, 14, 5]",0.9111111,0.83555555


In [ ]:
complete_df = all_scores.merge(all_new_scores.rename(columns={'score': 'untrained_score'}), on=['n_way', 'k_shot', 'n_query', 'n_iter', 'classes_to_sample_from'])
complete_df

,n_way,k_shot,n_query,n_iter,classes_to_sample_from,score,untrained_score
0,2,2,5,51,"[1, 16]",0.7966667,0.62333333
1,2,3,5,51,"[3, 16]",1.0,0.71000004
2,2,4,5,51,"[9, 4]",0.8833333,0.5966667
3,2,5,5,51,"[14, 0]",0.99,0.99
4,2,6,5,51,"[9, 4]",0.8833333,0.59333336
5,3,2,5,51,"[16, 1, 17]",0.87111115,0.57111114
6,3,3,5,51,"[9, 17, 4]",0.7022222,0.4577778
7,3,4,5,51,"[5, 3, 16]",0.90000004,0.6111111
8,3,5,5,51,"[14, 9, 5]",0.9266667,0.7777778
9,3,6,5,51,"[17, 14, 5]",0.9111111,0.83555555


In [ ]:
complete_df.to_csv('/content/drive/MyDrive/Colab Notebooks/few shot learning/few_shot_untrained_scores_complete_df2.csv')